In [1]:
!pip install pypdf
!pip install langchain
!pip install google-generativeai
!pip install langchain_google_genai
!pip install chromadb


  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/c9/d1/450b19bbdbb2c802f554312c62ce2a2c0d8744fe14735bc70ad2803578c7/pypdf-4.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/290.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/290.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/290.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/290.4 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/290.4 kB 163.8 kB/s eta 0:00:02
   ---- ---------------------------------- 30.7/290.4 kB 163.8 kB/s eta 0:00:02
   ----- --------------------------------- 41.0/290.4 kB 140.3 kB/s eta 0:00:02
   -------- ------------------------------ 61.4/290.4 kB 192.5 kB/s eta 0:00:02
   ---------- ---------------------------- 81.9/290.4 kB 241.3 kB/s eta 0:00:01
   ------------ -------------------------- 92.2/290.4 kB 238.1 kB/s eta 0:00:01
   -------------- --------------------

In [59]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Setup API Key
f = open(r"C:\Users\admin\Desktop\internship\RAG\api_key.txt")
GOOGLE_API_KEY = f.read()

chat_model = ChatGoogleGenerativeAI(google_api_key=GOOGLE_API_KEY, model="gemini-1.5-pro-latest")

In [61]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"C:\Users\user\Desktop\Internship\RAG\2404.07143.pdf")

data = loader.load_and_split()

data

[Document(page_content='Preprint. Under review.\nLeave No Context Behind:\nEfficient Infinite Context Transformers with Infini-attention\nTsendsuren Munkhdalai, Manaal Faruqui and Siddharth Gopal\nGoogle\ntsendsuren@google.com\nAbstract\nThis work introduces an efficient method to scale Transformer-based Large\nLanguage Models (LLMs) to infinitely long inputs with bounded memory\nand computation. A key component in our proposed approach is a new at-\ntention technique dubbed Infini-attention. The Infini-attention incorporates\na compressive memory into the vanilla attention mechanism and builds\nin both masked local attention and long-term linear attention mechanisms\nin a single Transformer block. We demonstrate the effectiveness of our\napproach on long-context language modeling benchmarks, 1M sequence\nlength passkey context block retrieval and 500K length book summarization\ntasks with 1B and 8B LLMs. Our approach introduces minimal bounded\nmemory parameters and enables fast strea

In [62]:
from langchain_text_splitters import NLTKTextSplitter

text_splitter = NLTKTextSplitter(chunk_size=500, chunk_overlap=100)

chunks = text_splitter.split_documents(data)

print(len(chunks))

print(type(chunks[0]))

Created a chunk of size 568, which is longer than the specified 500
Created a chunk of size 506, which is longer than the specified 500
Created a chunk of size 633, which is longer than the specified 500


110
<class 'langchain_core.documents.base.Document'>


In [63]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(google_api_key=GOOGLE_API_KEY, model="models/embedding-001")

In [64]:
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db_")

db.persist()

In [65]:
db_connection = Chroma(persist_directory="./chroma_db_", embedding_function=embedding_model)

In [66]:
retriever = db_connection.as_retriever(search_kwargs={"k": 5})

print(type(retriever))

<class 'langchain_core.vectorstores.VectorStoreRetriever'>


In [80]:
user_input = "What is Scaled Dot-product attention?"

In [81]:
retrieved_docs = retriever.invoke(user_input)

In [82]:
len(retrieved_docs)

5

In [83]:
print(retrieved_docs[0].page_content)

We
compute the standard causal dot-product attention context within each segment.

So the
dot-product attention computation is local in a sense that it covers a total Nnumber of
tokens of the current segment with index S(Nis the segment length).

The local attention (Dai et al., 2019), however, discards the attention states of the previous
segment when processing the next one.


In [84]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages([

    SystemMessage(content="""You are a Helpful AI Bot. 
    You take the context and question from user. Your answer should be based on the specific context."""),

    HumanMessagePromptTemplate.from_template("""Aswer the question based on the given context.
    Context:
    {context}
    
    Question: 
    {question}
    
    Answer: """)
])

In [85]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [86]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_template
    | chat_model
    | output_parser
)

In [87]:
response = rag_chain.invoke("What is Scaled Dot-product attention?")

response

'## Scaled Dot-Product Attention Explained\n\nBased on the context you provided, **Scaled Dot-Product Attention** is a specific mechanism within the broader concept of Multi-Head Attention, which is a crucial building block in Large Language Models (LLMs). Here\'s a breakdown:\n\n**Core Idea:**\n\nAt its heart, Scaled Dot-Product Attention involves calculating the attention scores between different elements in a sequence (e.g., words in a sentence). These scores determine how much each element should "attend" to other elements when processing information. \n\n**Mechanics:**\n\n1. **Dot Product:** The attention scores are computed using the dot product between the query (Q) and key (K) vectors associated with each element. This essentially measures the similarity between elements.\n\n2. **Scaling:** The dot product results are scaled down by the square root of the dimension of the key vectors. This scaling prevents the scores from becoming too large, especially with high-dimensional vec

In [88]:
from IPython.display import Markdown as md

md(response)

## Scaled Dot-Product Attention Explained

Based on the context you provided, **Scaled Dot-Product Attention** is a specific mechanism within the broader concept of Multi-Head Attention, which is a crucial building block in Large Language Models (LLMs). Here's a breakdown:

**Core Idea:**

At its heart, Scaled Dot-Product Attention involves calculating the attention scores between different elements in a sequence (e.g., words in a sentence). These scores determine how much each element should "attend" to other elements when processing information. 

**Mechanics:**

1. **Dot Product:** The attention scores are computed using the dot product between the query (Q) and key (K) vectors associated with each element. This essentially measures the similarity between elements.

2. **Scaling:** The dot product results are scaled down by the square root of the dimension of the key vectors. This scaling prevents the scores from becoming too large, especially with high-dimensional vectors, which could lead to unstable gradients during training.

3. **Softmax:** The scaled scores are then passed through a softmax function, which normalizes them into a probability distribution. This ensures that the attention weights for each element sum to 1.

4. **Weighted Sum:** Finally, the attention weights are used to create a weighted sum of the value (V) vectors associated with each element. This weighted sum represents the context vector, which captures the relevant information from the surrounding elements.

**Benefits:**

* **Efficiency:** Scaled Dot-Product Attention is computationally efficient compared to other attention mechanisms.
* **Parallelization:** The dot product and other operations can be easily parallelized, making it suitable for hardware acceleration.

**Contextual Significance:**

The provided text highlights that the attention computation is performed within segments, meaning it focuses on a local context rather than the entire input sequence. This approach can be more efficient for long sequences. However, the text also contrasts this with the local attention mechanism (Dai et al., 2019), which discards previous segment information, potentially losing valuable context. 
